Osnabrück University - Computer Vision (Winter Term 2020/21) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack, Ludwig Schallner, Artem Petrov

# Exercise Sheet 8: Compression

## Introduction

This week's sheet should be solved and handed in before the end of **Saturday, January 09, 2021**. If you need help (and Google and other resources were not enough), feel free to contact your groups' designated tutor or whomever of us you run into first. Please upload your results to your group's Stud.IP folder.

## Assignment 1: Redundancy and compression [4 Points]


**a)** Explain in your own words the different types of redundancy mentioned on (CV-10 slide 3). How can you check for each of these types of redundancy?

- **coding redundancy**:
    - coding scheme for the gray values encodes more values than necessary (part of the image)
    - check by computing entropy (information content)
- **interpixel redundancy**:
    - neighboring pixels have identical gray / color values
    - check by looking for identical gray /color values of neighboring pixels (homogeneous regions)
- **psychovisual redundancy**:
    - information that is not needed for recognition (by humans)
    - check e.g. by comparing pixel intensity range with recognizable levels of intensity

**b)** Explain the differences between lossless and lossy compression. Name examples for both of them. Sketch application scenarios.

- **lossless**: Only removes truly redundant information (not losing anything).
    - example: Compression based on coding- or interpixel redundancy (redundancy of the data)
        - e.g. Huffman coding or RLE
- **lossy**: If lossless compression is not sufficient, the information to be lost should be chosen in a way that does not cause any problems such as worse recognizability.
    - example: Compression based on psychovisual redundancy (redundancy of the information)
        - e.g. orthogonal transformations to divide data into psychovisually important / unimportant parts + subsequent compression

Therefore, lossy compression is not only removing true redundancies, but also information that is not that important.

## Assignment 2: Entropy based compression [8 Points]


**a)** Explain the idea of Huffman coding. What is the maximal compression factor that can be achieved for a given image? Load an image and compute that value (you may use `dolly.png` as an example. Make sure to load as 8-bit gray scale image).

**Huffman-Coding Steps**
- compute normalized histogram
- order symbols according to probability
- generate tree
    - merge the two least likely symbols
    - repeat until only two symbols remain
- start from the back and generate a prefix-free code according to the probability

The result is a coding scheme that assigns the shortest codes to the most likely signals and the longest codes to the least likely ones,
thereby removing the coding redundancy.  
The theoretical maximum compression factor is given by $\frac{\#bits}{entropy}$, which intuitively makes sense, because if the entropy is high ('chaotic' image ~uniformly distributed gray values), there is not much to be compressed (low compression factor). If, on the other hand, the entropy is low (only a few different gray values), there is a lot to be compressed.

In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt

img = imageio.imread('images/dolly.png', pilmode='L')
# test
#img = np.array([[1, 1, 2, 2], [2, 3, 2, 3], [2, 2, 2, 2], [3, 2, 3, 3]])

# YOUR CODE HERE

def entropy(img):
    """
    Compute the entropy for a given image.
    Args:
        img (ndarray): The grayscale image to compute the entropy for.
    
    Returns:
        img_entropy (float): The entropy of the image. 
    """
    hist = img.flatten()
    entropy = 0
    for i in range(256):
        counter = np.count_nonzero(hist == i)
        prob = counter / (img.shape[0] * img.shape[1])
        if prob > 0.0:
            entropy += prob * np.log2(prob)
    return -entropy

comp_fac = (8 / entropy(img))
print('theoretical maximum compression factor:', comp_fac)

plt.imshow(img, cmap='gray')

**b)** Now compute the relative frequencies (normalized histogram) of the image and generate an (approximately) balanced tree, as described in (CV-10 slide 6). *Hint:* you may use Python tuples as building blocks of a tree. Every non-leaf node is a pair `(left, right)` where `left` and `right` are the left and right subtrees, respectively (of course you are free to choose another approach if you prefer to do so).

In [ ]:
from collections import namedtuple
from sortedcontainers import SortedList  # conda install -c conda-forge sortedcontainers
from heapq import *

class Node:
    def __init__(self, frequency, subtree):
        self.frequency = frequency
        self.subtree = subtree

    def __lt__(self, other):
        return self.frequency < other.frequency

    def __str__(self):
        return "freq: " + str(self.frequency) + ", sub: " + str(self.subtree)

# first compute a histogram --> count how often each gray value is used
b, bins, patches = plt.hist(img.flatten(), 256, (0, 255))
norm_hist = (b - np.min(b)) / (np.max(b) - np.min(b))

# nodes is a sorted list of sub-trees, each annotated by its cummulative relative frequency,
# i.e. each list item is a pair (frequency, subtree) - lowest frequencies come first
nodes = SortedList([Node(frequency, i) for i, frequency in enumerate(norm_hist)], key=lambda x: x.frequency)

tree = list(nodes)
heapify(tree)

while len(tree) > 1:
    left = heappop(tree)
    right = heappop(tree)
    heappush(tree, Node(left.frequency + right.frequency, (left, right)))

**c)** Now create a prefix free code from this tree, by traversing it following the idea sketched in (CV-10 slide 7). *Hint:* if you used the tuple representation recommended in (b), you can use `isinstance(node, tuple)` to check if `node` is an inner node or a leaf.

In [ ]:
codes = ['' for char in range(256)]

# recursively traverse the tree
def assign_codes(codes, tree, prefix=''):
    if isinstance(tree.subtree, tuple):
        assign_codes(codes, tree.subtree[0], prefix + '0')
        assign_codes(codes, tree.subtree[1], prefix + '1')
    else:
        codes[tree.subtree] = prefix

assign_codes(codes, tree[0])

**d)** Compute the compression ratio that you can achieve with that code. Compare this with the maximal value computed in part (a). Explain your observation.

In [ ]:
required_memory = 0
constant_mem = 0
for value, code in enumerate(codes):
    required_memory += len(code) * norm_hist[value]
    constant_mem += 8 * norm_hist[value]

print('required memory: {:.2f}'.format(required_memory))
print('memory for constant code length: {:.2f}'.format(constant_mem))
print("compression ratio:", (constant_mem) / required_memory)

The maximum compression factor computed in (a) was $\approx 1.077$, so we end up with a compression that is pretty close to the theoretical optimum.  
The entropy is a measure of the smallest codeword length that is theoretically possible for the given symbols. Therefore, Huffman coding performs very well.

## Assignment 3: Run length encoding [8 Points]


**a)** Explain the idea of *run length encoding*. What are advantages and disadvantages? In what situations should it be applied?

RLE removes redundancy caused from identical gray / color values of neighboring pixels.  
The image is coded line by line and segments of lines with identical gray or color value are coded in the form $(number, value)$.

**Disadvantages**:
- in the worst case, the amount of data is doubled (just each pixel together with the number $1$)
- inefficient for noisy images because the runs are very short

**Advantages**:
- is able to reduce redundancy by grouping homogeneous areas (if there are any)

It obviously should be applied for images with large (truly) homogeneous areas.

**b)** Analyze the run lengths in a gray scale image (8 bit = 256 gray values) by counting the number of runs and the average run length and displaying a histogram of the run lengths. What do you observe? Can you benefit from run length encoding here? (you may use `dolly.png` as an example again, but you may also experiment with other images. Make sure to load it as 8-bit gray scale image).

In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt

img = imageio.imread('images/dolly.png', pilmode='L')
img_2 = imageio.imread('images/8bitmario.jpeg', pilmode='L')

# slide example
# img = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])
# img  = np.array([[1, 1, 1, 2, 3, 3, 4, 4, 4, 4], [1, 1, 2, 2, 3, 4, 4, 4, 4, 4], [1, 2, 3, 4, 4, 4, 5, 5, 5, 5]])

# YOUR CODE HERE

def plot_run_len_hist(runs):
    bins = np.unique(runs)
    hist = [runs.count(b) for b in bins]
    plt.bar(bins, hist)

def get_runs(img):
    runs = []
    for line in img:
        cnt = 0
        val_before = -1
        line_runs = []
        for val in line:
            if val != val_before:
                if val_before != -1:
                    line_runs.append((cnt, val_before))
                cnt = 1
                val_before = val
            else:
                cnt += 1
        if cnt != 1:
            line_runs.append((cnt, val_before))
        runs.append(line_runs)
    return runs

def get_num_of_runs(img):
    runs = get_runs(img)
    return np.sum([len(line) for line in runs])

def get_avg_run_len(img):
    runs = get_runs(img)
    return np.average([run[0] for line in runs for run in line])

print('num of runs (original img):', get_num_of_runs(img))
print('avg run length (original img):', get_avg_run_len(img))
print('num of runs (mario img):', get_num_of_runs(img_2))
print('avg run length (mario img):', get_avg_run_len(img_2))

plt.figure(figsize=(20, 4))
plt.gray()

runs = get_runs(img)
plt.subplot(1, 4, 1); plt.title('original image'); plt.imshow(img)
plt.subplot(1, 4, 2); plt.title('run length histogram')
plot_run_len_hist([run[0] for line in runs for run in line])

runs = get_runs(img_2)
plt.subplot(1, 4, 3); plt.title('mario image'); plt.imshow(img_2)
plt.subplot(1, 4, 4); plt.title('run length histogram')
plot_run_len_hist([run[0] for line in runs for run in line])

We cannot really benefit from run length encoding for `dolly.png` since it has an avg run length of $1.12$, which means that almost  
every run has a length of $1$ and therefore the amount of data is roughly doubled, which is exactly the opposite of what one wants to achieve when using it.  
The problem with the image is that it does not contain large (truly) homogenous areas. There is noise and even the homogenous areas do not contain exactly the same gray values.

It's a lot better for the mario image that contains large truly homogenous areas and leads to a avg run length of $127.81$.

**c)** Now consider the individual bit planes. First display the bit planes as in (CV-10 slide 10ff). What do you observe? Apply your analysis from part (b) to each bitplane.

In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt

img = imageio.imread('images/dolly.png', pilmode='L')

# YOUR CODE HERE

bin_rep_img = [np.binary_repr(img[i][j], width=8) for i in range(img.shape[0]) for j in range(img.shape[1])]

def get_bit_plane(bit):
    return (np.array([int(i[bit]) for i in bin_rep_img], dtype=np.uint8) * (2 ** (7 - bit))).reshape(img.shape)

def get_run_counts(runs):
    return [run[0] for line in runs for run in line]

plt.figure(figsize=(16, 6))
plt.gray()
plt.subplot(2, 4, 1); plt.title('1st bit'); plt.imshow(get_bit_plane(0))
plt.subplot(2, 4, 2); plt.title('2nd bit'); plt.imshow(get_bit_plane(1))
plt.subplot(2, 4, 3); plt.title('3rd bit'); plt.imshow(get_bit_plane(2))
plt.subplot(2, 4, 4); plt.title('4th bit'); plt.imshow(get_bit_plane(3))
plt.subplot(2, 4, 5); plt.title('5th bit'); plt.imshow(get_bit_plane(4))
plt.subplot(2, 4, 6); plt.title('6th bit'); plt.imshow(get_bit_plane(5))
plt.subplot(2, 4, 7); plt.title('7th bit'); plt.imshow(get_bit_plane(6))
plt.subplot(2, 4, 8); plt.title('8th bit'); plt.imshow(get_bit_plane(7))

plt.figure(figsize=(16, 6))
plt.gray()
plt.subplot(2, 4, 1); plt.title('RLE 1st bit'); runs = get_runs(get_bit_plane(0)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 2); plt.title('RLE 2nd bit'); runs = get_runs(get_bit_plane(1)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 3); plt.title('RLE 3rd bit'); runs = get_runs(get_bit_plane(2)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 4); plt.title('RLE 4th bit'); runs = get_runs(get_bit_plane(3)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 5); plt.title('RLE 5th bit'); runs = get_runs(get_bit_plane(4)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 6); plt.title('RLE 6th bit'); runs = get_runs(get_bit_plane(5)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 7); plt.title('RLE 7th bit'); runs = get_runs(get_bit_plane(6)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 8); plt.title('RLE 8th bit'); runs = get_runs(get_bit_plane(7)); plot_run_len_hist(get_run_counts(runs))

It works very well for the first few bit planes and gets increasingly worse towards the least significant bit plane.  
That's not surprising since the first bit plane only distinguishes pixels from the lower and upper half of the range of possible gray values,  
which means that small changes in the gray values between neighboring pixels still belong to the same run here.  
In contrast, on the least significant bit plane, we again have a lot of change.

**d)** Explain the idea of the *Gray code*. Why is it better suited for run length encoding? Compute a Gray code for a 256 bit image and recode the image `dolly.png`. Then analyze the run lengths of the individual bit planes of the recoded image.

Gray code is an ordering of the binary system such that two successive values differ in only one bit (Hamming distance $1$).  
The gray code image does lead to an improvement, although it's not that big.  
The enhancement results from the fact that successive number differ in only one bit, which means that only one bit plane is disturbed.  

In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt

img = imageio.imread('images/dolly.png', pilmode='L')

# YOUR CODE HERE

bin_rep_img = [np.binary_repr(img[i][j], width=8) for i in range(img.shape[0]) for j in range(img.shape[1])]

# Binary to Gray conversion : 
#   - the most significant bit of the gray code is always equal to the MSB of the given binary code
#   - other bits of the output gray code can be obtained by XORing binary code bit at that index and previous index

def bin_to_gray(pixel):
    gray = ""
    # MSB of gray code is same
    gray += pixel[0]
    for i in range(1, len(pixel)):
        # XOR of previous bit with current bit
        gray += str(int(pixel[i - 1]) ^ int(pixel[i]))
    return gray

gray_code_img = [bin_to_gray(pixel) for pixel in bin_rep_img]
bin_rep_img = gray_code_img

plt.figure(figsize=(16, 6))
plt.gray()
plt.subplot(2, 4, 1); plt.title('1st bit'); plt.imshow(get_bit_plane(0))
plt.subplot(2, 4, 2); plt.title('2nd bit'); plt.imshow(get_bit_plane(1))
plt.subplot(2, 4, 3); plt.title('3rd bit'); plt.imshow(get_bit_plane(2))
plt.subplot(2, 4, 4); plt.title('4th bit'); plt.imshow(get_bit_plane(3))
plt.subplot(2, 4, 5); plt.title('5th bit'); plt.imshow(get_bit_plane(4))
plt.subplot(2, 4, 6); plt.title('6th bit'); plt.imshow(get_bit_plane(5))
plt.subplot(2, 4, 7); plt.title('7th bit'); plt.imshow(get_bit_plane(6))
plt.subplot(2, 4, 8); plt.title('8th bit'); plt.imshow(get_bit_plane(7))

plt.figure(figsize=(16, 6))
plt.gray()
plt.subplot(2, 4, 1); plt.title('RLE 1st bit'); runs = get_runs(get_bit_plane(0)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 2); plt.title('RLE 2nd bit'); runs = get_runs(get_bit_plane(1)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 3); plt.title('RLE 3rd bit'); runs = get_runs(get_bit_plane(2)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 4); plt.title('RLE 4th bit'); runs = get_runs(get_bit_plane(3)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 5); plt.title('RLE 5th bit'); runs = get_runs(get_bit_plane(4)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 6); plt.title('RLE 6th bit'); runs = get_runs(get_bit_plane(5)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 7); plt.title('RLE 7th bit'); runs = get_runs(get_bit_plane(6)); plot_run_len_hist(get_run_counts(runs))
plt.subplot(2, 4, 8); plt.title('RLE 8th bit'); runs = get_runs(get_bit_plane(7)); plot_run_len_hist(get_run_counts(runs))